Обработка отчета по проводкам из 1С

In [2]:
import pandas as pd
import numpy as np

In [4]:
# read excel file downloaded from 1C accounting system
# delete null rows and columns
filename = 'Им_Отчет_по_проводкам_за_01_10_2022_24_10_2022.xlsx'
df = pd.read_excel(filename, 
                    skiprows=5, 
                    header=0, 
                    names = ['date', 'document', 'dr_description', 'cr_description', 
                             'dr_account', 'dr_amount', 'to_del', 'cr_account', 'cr_amount'],
                 )
df.drop('to_del', axis=1, inplace=True, errors='ignore')
df.drop(0, axis=0, inplace=True, errors='ignore')
df.drop(len(df.index), axis=0, inplace=True, errors='ignore') #delete last row with total sum

In [5]:
#df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True, utc=True, errors='ignore')
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

In [6]:
# divide column with 3 descriptions into 3 columns
df[['dr_1', 'dr_2', 'dr_3']] = df['dr_description'].str.split('\n',expand=True)
df[['cr_1', 'cr_2', 'cr_3']] = df['cr_description'].str.split('\n',expand=True)

In [5]:
#add group company name
df['group_company'] = 'Company_1' 

In [14]:
# add short account without subaccounts
df['dr_acc_short'] = df['dr_account'].str.split('.').str[0]
df['cr_acc_short'] = df['cr_account'].str.split('.').str[0]
df['dr_acc_short'] = df['dr_acc_short'].astype('int')
df['cr_acc_short'] = df['cr_acc_short'].astype('int')

In [16]:
# classify transactions
conditions = [
    (df.dr_acc_short == 26),
    (df.dr_acc_short == 91),
    (df.dr_acc_short != 51) & (df.cr_acc_short == 70),
    (df.dr_acc_short == 62) | (df.dr_acc_short == 90),
    (df.dr_acc_short == 51) & (df.dr_acc_short == 62)
]
 
values = ['admin_expenses', 'other_expenses', 'payroll', 'revenue_accrual', 'revenue_cash']

df['acc_type'] = np.select(conditions, values)


In [13]:
df.dtypes #check data types

date              datetime64[ns]
document                  object
dr_description            object
cr_description            object
dr_account                object
dr_amount                float64
cr_account                object
cr_amount                float64
year                       int64
month                      int64
dr_1                      object
dr_2                      object
dr_3                      object
cr_1                      object
cr_2                      object
cr_3                      object
group_company             object
dr_acc_short               int32
cr_acc_short              object
acc_type                  object
dtype: object